In [1]:
import numpy as np
import plotly.graph_objs as go
from ipywidgets import widgets
from static_allelic_freq import get_static_plot
from filenames_helper.collect_filenames import collect_filenames


/Users/cmesser/PycharmProjects/plotly-interactive/venv/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
# define column_names

# define workspace name
workspace = 'broad-firecloud-ibmwatson/Getz_Ebert_IBM_13-583_Exomes_Analysis'

In [3]:
## Get list of maf filenames

In [5]:
### From Terra
maf_list = collect_filenames(workspace=workspace, set_type='pair',
                             set_name='all_samples_except_MGH_484718',
                             column_name='mutation_validator_validated_maf')

KeyError: 'mutation_validated_validator_maf'

In [ ]:
### Or From File
# Can pull filenames from a text file
# Can be useful to specify a mix of local, downloaded mafs and urls hosted in a bucket

# maf_list = collect_filenames(filepath='/Users/me/my_documents/my_directory/my_file.txt')

In [6]:
# get basic static plot
fig, samples, gnomad_min, gnomad_max, column_names = get_static_plot(maf_list, workspace)

# parameters
filter_disc_num = 20

NameError: name 'maf_list' is not defined

# Add Widgets

In [ ]:
gnomAD_slider = widgets.FloatSlider(
    value=0,
    min=0,
    max=max(samples[column_names.get('gnomad_freq')]),
    step=max(samples[column_names.get('gnomad_freq')]) / filter_disc_num,
    disabled=False,
    description='GnomAD Frequency Filter: ',
    readout_format='.5f'
)

gnomAD_nan = widgets.Checkbox(
    value=True,
    description='Allow NaN gnomAD points',
    disabled=False,
    indent=False
)

all_mutations = samples[column_names.get('variant_class')].unique().tolist()
mutation_select = widgets.SelectMultiple(
    options=all_mutations,
    value=all_mutations,
    description='Mutation Types:  ',
    disabled=False
)

ipython_fig = go.FigureWidget(fig)

df_w_filter = samples.copy()

def response(change):
    filter_list = [m in mutation_select.value and
                   ((g >= gnomAD_slider.value and not np.isnan(g))
                   or (np.isnan(g) and gnomAD_nan.value))
                   for m, g in zip(samples[column_names.get('variant_class')],
                                   samples[column_names.get('gnomad_freq')])]
    df_w_filter['filter'] = filter_list
    updated_sum_data = df_w_filter.groupby(level=[0], sort=False).sum()['filter'].values.flatten()

    temp_df = samples[filter_list]
    y = temp_df[column_names.get('tumor_freq')]
    x = temp_df['x_jitter']
    t_count = temp_df['t_ref_count'].values + temp_df['t_alt_count'].values
    with ipython_fig.batch_update():
        ipython_fig.data[0].x = x
        ipython_fig.data[0].y = y
        ipython_fig.data[0].marker['color'] = temp_df[column_names.get('gnomad_freq')]
        ipython_fig.data[0].marker['cmin'] = gnomad_min
        ipython_fig.data[0].marker['cmax'] = gnomad_max
        ipython_fig.data[0].customdata = np.stack((temp_df[column_names.get('hugo')].tolist(),
                                                   temp_df['Protein_Change'].tolist(),
                                                   t_count), axis=-1)
        ipython_fig.data[1].z = updated_sum_data
        for x_val, this_sum in enumerate(updated_sum_data):
            ipython_fig['layout']['annotations'][x_val].update(text=str(this_sum))

mutation_select.observe(response, names="value")
gnomAD_slider.observe(response, names="value")
gnomAD_nan.observe(response, names="value")

widgets.VBox([mutation_select, ipython_fig, gnomAD_slider, gnomAD_nan])